In [ ]:
#================================================
#DEBUG TOGGLE SWITCH

def d(*args, **kwargs):
    if d.ON:
        print(*args, **kwargs)

d.ON = False          # <- module-local switch

##example usage
#d("Initial x shape:", x.shape)
#================================================


#===============================================================


#===============================================================
#================================================
#WATCH OUT! ONLY EXECUTABLE IN THE ACTUAL NOTEBOOK!!!! SO MAKE SURE IT'S COMMENTED WHEN YOU EXPORT IS AS .py!!!! next time do that thing you do for dummy tests (name = main)
"""
try:
    import segmentation_models_pytorch as smp
except ImportError:
    !pip install -q segmentation-models-pytorch
    !pip install -q timm
    import segmentation_models_pytorch as smp
"""


#========================================================

import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

#======================================================

# CLASS DEFINITION
class DeepLabDualHead(nn.Module):
  def __init__(self, n_classes=7):
    super().__init__()

    self.base = smp.DeepLabV3Plus( #base model DeepLabV3+ kept as it is for ENCODER+DECODER, but gotta override the head to introduce novelty
       encoder_name="resnet50",
       encoder_weights="imagenet",
       in_channels=3,
       classes=n_classes #will get overridden by novelty trait planned
    )

    self.base.classifier = nn.Identity() # removing the built-in classification head and just keepin encoder/decoder arch

    #SEGMENTATION HEAD
    self.head_softmax = nn.Conv2d(256, n_classes, kernel_size=1)  #256 is the number of input channels in the layer, n_classes as the number of predictions we want for each pixel (probability for each class ofc), 1 as k.size means each convolution is computed independently pixel for pixel

    #OBJECTNESS HEAD
    self.head_sigmoid = nn.Sequential(
        nn.Conv2d(256, 1, kernel_size=1), #256 is the number of input channels in the layer, 1 as the single prediction we want for each pixel (obj/!obj), 1 as k.size means each convolution is computed independently pixel for pixel
        nn.Sigmoid()
    )

  def forward(self, x):
    features = self.base.encoder(x) #input goes through encoder, features'lists get produced
    decoder_output = self.base.decoder(features) #features'list go through decoder, has to find out the semantic output EDIT (error in training loop): DeepLabV3PlusDecoder takes 2 positional arguments but 7 were given, so i can't destructure the list and pass separated arguments, but the whole one as it is

    #SEMANTIC PREDICTION OUTPUT CONSTRUCTION
    softmax_output = self.head_softmax(decoder_output)

    #OBJECTNESS PREDICTION OUTPUT CONSTRUCTION
    sigmoid_output = self.head_sigmoid(decoder_output)

    return softmax_output, sigmoid_output

#============================================================


#==============================================================
#way to export it
def get_model():
  return DeepLabDualHead()
#==============================================================

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.5 MB/s eta 0:00:00
